# 자연어처리 : 기말고사 대체 프로젝트
# 2019312072 정주호
# ACCURACY : 72.4%
# 전체 런타임 : 15분

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("/content/ttrain.csv")

In [3]:
train[:5]

,Unnamed: 0,Unnamed: 1
0,title,label
1,유튜브 내달 2일까지 크리에이터 지원 공간 운영,IT과학
2,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화
3,내년부터 국가RD 평가 때 논문건수는 반영 않는다,IT과학
4,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,IT과학


In [4]:
train = train.rename(columns=train.iloc[0])
train = train.drop(train.index[0])

In [5]:
train[:5]

,title,label
1,유튜브 내달 2일까지 크리에이터 지원 공간 운영,IT과학
2,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화
3,내년부터 국가RD 평가 때 논문건수는 반영 않는다,IT과학
4,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,IT과학
5,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화


In [6]:
train['title'] = train['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train['title'].replace('', np.nan, inplace=True)
print(train.isnull().sum())

title    0
label    0
dtype: int64


In [7]:
train[:5]

,title,label
1,유튜브 내달 일까지 크리에이터 지원 공간 운영,IT과학
2,어버이날 맑다가 흐려져남부지방 옅은 황사,생활문화
3,내년부터 국가 평가 때 논문건수는 반영 않는다,IT과학
4,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,IT과학
5,회색인간 작가 김동식 양심고백 등 새 소설집 권 출간,생활문화


In [8]:
index_dic = {'경제' :0, '사회':1, '생활문화':2, '세계':3, '스포츠':4, '정치':5, 'IT과학':6}

In [9]:
Y_train = []
for i in train['label']:
  Y_train.append(index_dic[i])

In [10]:
Y_train[:5]

[6, 2, 6, 6, 2]

In [11]:
Y_train.count(6)

7852

In [12]:
test = pd.read_csv("/content/ttest.csv")

test = test.rename(columns=test.iloc[0])
test = test.drop(test.index[0])

test['title'] = test['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['title'].replace('', np.nan, inplace=True)
print(test.isnull().sum())

title    0
label    0
dtype: int64


In [13]:
Y_test = []
for i in test['label']:
  Y_test.append(index_dic[i])

In [14]:
stopwords = [ '의', '가', '이', '한', '도', '는', '은', '다', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '너','나','네', '들', '듯', '만', '지', '임', '게', '겜', '되','돼','음', '면']

In [15]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 44.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [16]:
import konlpy 
from konlpy.tag import Okt 
from tqdm import tqdm
okt = Okt() 

X_train = [] 
for sen in tqdm(train['title']): 
  temp = [] 
  temp = okt.morphs(sen, stem=True) 
  temp = [word for word in temp if not word in stopwords] 
  X_train.append(temp) 

X_test = [] 
for sen in tqdm(test['title']): 
  temp = [] 
  temp = okt.morphs(sen, stem=True)
  temp = [word for word in temp if not word in stopwords] 
  X_test.append(temp)


100%|██████████| 9107/9107 [00:11<00:00, 795.01it/s]


In [17]:
X_train[:5]

[['유튜브', '내달', '일', '까지', '크리에이터', '지원', '공간', '운영'],
 ['어버이날', '맑다', '흐려지다', '남부', '지방', '옅다', '황사'],
 ['내년', '부터', '국가', '평가', '때', '논문', '건수', '반영', '않다'],
 ['김명자', '신임', '과총', '회장', '원로', '젊다', '과학자', '지혜', '모으다', '것'],
 ['회색',
  '인간',
  '작가',
  '김',
  '동식',
  '양심',
  '고백',
  '등',
  '새',
  '소설',
  '집',
  '권',
  '추다',
  '간']]

In [18]:
X_test[:3]

[['억원', '무', '이자', '융자', '되다', '천만원', '이사비', '안되다'],
 ['왜', '수', '소', '충전', '소만', '더', '멀리', '떨어지다', '하나', '경연', '규제', '개혁', '건의'],
 ['항', '응고', '제', '성분', '코로나', '효과', '세포', '실험', '서', '확인']]

In [19]:
print(len(X_test))
print(len(X_train))
print(len(Y_test))
print(len(Y_train))

9107
45678
9107
45678


In [20]:
print("train title 최대 길이 : ", max(len(token) for token in X_train))
print("train title 평균 길이 : ", sum(map(len, X_train))/ len(X_train)) 

train title 최대 길이 :  17
train title 평균 길이 :  8.750492578484172


In [21]:
print("test title 최대 길이 : ", max(len(token) for token in X_test))
print("test title 평균 길이 : ", sum(map(len, X_test))/ len(X_test)) 

test title 최대 길이 :  16
test title 평균 길이 :  8.98814099044691


In [22]:
from keras.preprocessing.text import Tokenizer 
tokenizer = Tokenizer(20000) 
tokenizer.fit_on_texts(X_train) 
x_train = tokenizer.texts_to_sequences(X_train) 
x_test = tokenizer.texts_to_sequences(X_test)


In [23]:
x_train[:3]

[[1361, 226, 5, 40, 6387, 89, 1721, 396],
 [10884, 1305, 8407, 716, 1032, 7591, 2682],
 [139, 50, 290, 487, 283, 2774, 5550, 2345, 109]]

In [24]:
x_test[:5]

[[46, 762, 2007, 16505, 19, 1037, 3649],
 [1158, 70, 298, 2351, 56, 9371, 1769, 248, 3538, 363, 403, 3337],
 [2712, 47, 13347, 647, 1993, 1240, 3, 338],
 [574, 355, 405, 2322, 8318, 105, 21, 183, 1457],
 [1492, 2, 66, 15853, 485]]

In [25]:
print(len(x_test))
print(len(x_train))

9107
45678


In [26]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)
print(y_train)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [68]:
print(len(y_test))
print(len(y_train))

9107
45678


In [73]:
from keras.layers import Embedding, Dense, LSTM ,Bidirectional, SimpleRNN, GRU, BatchNormalization, Dropout
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences


x_train = pad_sequences(x_train, maxlen=20)
x_test = pad_sequences(x_test, maxlen=20)

model = Sequential() 
model.add(Embedding(20000, 200, input_length=20)) 
model.add(GRU(64, dropout=0.8))
#model.add(BatchNormalization())
model.add(Dense(32))
model.add(Dense(7,activation='softmax')) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(x_train, y_train, epochs=6, batch_size=10, validation_split=0.2)

Epoch 1/6
3655/3655 [==============================] - 204s 55ms/step - loss: 0.7764 - accuracy: 0.7310 - val_loss: 0.2879 - val_accuracy: 0.9155
Epoch 2/6
3655/3655 [==============================] - 201s 55ms/step - loss: 0.3715 - accuracy: 0.8830 - val_loss: 0.2505 - val_accuracy: 0.9251
Epoch 3/6
3655/3655 [==============================] - 201s 55ms/step - loss: 0.2757 - accuracy: 0.9102 - val_loss: 0.2252 - val_accuracy: 0.9304
Epoch 4/6
3655/3655 [==============================] - 201s 55ms/step - loss: 0.2200 - accuracy: 0.9290 - val_loss: 0.2196 - val_accuracy: 0.9321
Epoch 5/6
3655/3655 [==============================] - 202s 55ms/step - loss: 0.1803 - accuracy: 0.9414 - val_loss: 0.2474 - val_accuracy: 0.9306
Epoch 6/6
3655/3655 [==============================] - 201s 55ms/step - loss: 0.1581 - accuracy: 0.9507 - val_loss: 0.2457 - val_accuracy: 0.9296


In [74]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

predict = model.predict(x_test)
predict = np.argmax(predict,axis=1)
predict = to_categorical(predict)


print("accuracy : ",accuracy_score(y_test, predict))
print(classification_report(predict,y_test))

accuracy :  0.7243878335346436
              precision    recall  f1-score   support

           0       0.85      0.77      0.81      1862
           1       0.52      0.93      0.67      1870
           2       0.85      0.75      0.80      1556
           3       0.78      0.63      0.70       920
           4       0.94      0.84      0.89       635
           5       0.87      0.46      0.60      1210
           6       0.79      0.55      0.65      1054

   micro avg       0.72      0.72      0.72      9107
   macro avg       0.80      0.70      0.73      9107
weighted avg       0.78      0.72      0.73      9107
 samples avg       0.72      0.72      0.72      9107



In [75]:
print(model.evaluate(x_test,y_test))

285/285 [==============================] - 1s 5ms/step - loss: 1.0105 - accuracy: 0.7244
[1.0104869604110718, 0.7243878245353699]
